In [1]:
import pathlib
import numpy as np

# turn our training path into a Python path
data_dir = pathlib.Path("D:/KNUniversity/4 year/neural networks/food_classes/train/")

# created a list of class_names from the subdirectories
class_names = np.array(sorted([item.name for item in data_dir.glob('*')])) 
print(class_names)

['beignets' 'falafel' 'nachos']


In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Set the seed
tf.random.set_seed(42)

# Preprocess data (get all of the pixel values between 1 and 0, also called scaling/normalization)
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

# Setup the train and test directories
train_dir = "D:/KNUniversity/4 year/neural networks/food_classes/test/"
test_dir = "D:/KNUniversity/4 year/neural networks/food_classes/train/"

In [18]:
# Import data from directories and turn it into batches
train_data = train_datagen.flow_from_directory(train_dir,
                                               batch_size=32, # number of images to process at a time 
                                               target_size=(224, 224), # convert all images to be 224 x 224
                                               class_mode="categorical", # type of problem we're working on
                                               seed=42)

valid_data = valid_datagen.flow_from_directory(test_dir,
                                               batch_size=32,
                                               target_size=(224, 224),
                                               class_mode="categorical",
                                               seed=42)

Found 750 images belonging to 3 classes.
Found 225 images belonging to 3 classes.


In [2]:
# Create tensorboard callback (functionized because need to create a new one for each model)
import datetime
def create_tensorboard_callback(dir_name, experiment_name):
  log_dir = dir_name + "/" + experiment_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
  tensorboard_callback = tf.keras.callbacks.TensorBoard(
      log_dir=log_dir
  )
  print(f"Saving TensorBoard log files to: {log_dir}")
  return tensorboard_callback

In [5]:
import tensorflow_hub as hub

In [6]:
resnet_url="https://tfhub.dev/google/imagenet/resnet_v1_101/feature_vector/5"
efficientnet_url="https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"

In [7]:
IMAGE_SHAPE=(224,224)

In [13]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, MaxPool2D,Flatten,Dense

In [14]:
def create_BC_model(model_url):
  """Takes a TensorFlow Hub URL and creates a Keras Sequential model with it.
  
  Args:
    model_url (str): A TensorFlow Hub feature extraction URL.
    num_classes (int): Number of output neurons in output layer,
      should be equal to number of target classes, default 10.

  Returns:
    An uncompiled Keras Sequential model with model_url as feature
    extractor layer and Dense output layer with num_classes outputs.
  """
  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url,
                                           trainable=False, # freeze the underlying patterns
                                           name='feature_extraction_layer',
                                           input_shape=IMAGE_SHAPE+(3,)) # define the input image shape
  
  # Create our own model
  model = tf.keras.Sequential([
    feature_extractor_layer, # use the feature extraction layer as the base
    Dense(3, activation='softmax', name='output_layer') # create our own output layer      
  ])

  return model

In [15]:
resnet_model= create_BC_model(resnet_url)
resnet_model.compile(
    loss="categorical_crossentropy", 
    optimizer=tf.keras.optimizers.Adam(), 
    metrics=['accuracy']
)

In [19]:
resnet_model.fit(train_data, 
epochs=2, validation_data=valid_data, 
callbacks=[create_tensorboard_callback(dir_name="lab5", experiment_name="resnet")]
)

Saving TensorBoard log files to: lab5/resnet/20211124-233233
Epoch 1/2
24/24 [==============================] - 292s 12s/step - loss: 0.6209 - accuracy: 0.7253 - val_loss: 0.5162 - val_accuracy: 0.8044
Epoch 2/2
24/24 [==============================] - 292s 12s/step - loss: 0.2586 - accuracy: 0.9120 - val_loss: 0.3898 - val_accuracy: 0.8489


In [20]:
%reload_ext tensorboard

In [21]:
%tensorboard --logdir=lab5